In [2]:
import pandas as pd

In [3]:
ch = pd.read_csv('chemicals.csv')

In [4]:
ch.head()

,cws_name,chemical_species,contaminant_level,county,pws_id,pop_served,state,unit_measurement,value,year,fips,state_fips
0,CALIFORNIA WATER SERVICE - LIVERMORE,Uranium,Less than or equal MCL,Alameda,CA0110003,57200,California,micrograms/L,1.54,2000,6001,6
1,CITY OF LIVERMORE,Uranium,Less than or equal MCL,Alameda,CA0110011,26400,California,micrograms/L,1.64,2000,6001,6
2,CITY OF PLEASANTON,Uranium,Less than or equal MCL,Alameda,CA0110008,70600,California,micrograms/L,1.64,2000,6001,6
3,DUBLIN SAN RAMON SERVICES DISTRICT,Uranium,Less than or equal MCL,Alameda,CA0110009,72500,California,micrograms/L,1.64,2000,6001,6
4,NORRIS CANYON PROPERTY OWNERS ASSN,Uranium,Less than or equal MCL,Alameda,CA0103040,100,California,micrograms/L,4.51,2000,6001,6


In [5]:
ch.isnull().sum()

cws_name             0
chemical_species     0
contaminant_level    0
county               0
pws_id               0
pop_served           0
state                0
unit_measurement     0
value                0
year                 0
fips                 0
state_fips           0
dtype: int64

In [13]:
ch.describe()


,pop_served,value,year,fips,state_fips
count,10.000000,10.00000,10.000000,10.0,10.0
mean,70360.300000,1.82300,2000.200000,6001.0,6.0
std,102163.638383,1.10733,0.421637,0.0,0.0
min,40.000000,0.64000,2000.000000,6001.0,6.0
25%,630.000000,1.56500,2000.000000,6001.0,6.0
50%,41800.000000,1.64000,2000.000000,6001.0,6.0
75%,72025.000000,1.64000,2000.000000,6001.0,6.0
max,328325.000000,4.51000,2001.000000,6001.0,6.0


In [12]:
dr = pd.read_csv('droughts.csv')

In [8]:
dr.head()

,fips,county,state,none,d0,d1,d2,d3,d4,valid_start,valid_end
0,2013,Aleutians East Borough,AK,100,0,0,0,0,0,2017-12-26,2018-01-01
1,2013,Aleutians East Borough,AK,100,0,0,0,0,0,2017-12-19,2017-12-25
2,2013,Aleutians East Borough,AK,100,0,0,0,0,0,2017-12-12,2017-12-18
3,2013,Aleutians East Borough,AK,100,0,0,0,0,0,2017-12-05,2017-12-11
4,2013,Aleutians East Borough,AK,100,0,0,0,0,0,2017-11-28,2017-12-04


In [9]:
dr.isnull().sum()

fips           0
county         0
state          0
none           0
d0             0
d1             0
d2             0
d3             0
d4             0
valid_start    0
valid_end      0
dtype: int64

In [17]:
dr['valid_start'] = pd.to_datetime(dr['valid_start'])
dr['valid_end'] = pd.to_datetime(dr['valid_end'])

In [62]:
dr['duration'] = dr.valid_end - dr.valid_start
dr['duration_int'] = dr.duration.dt.days

In [27]:
dr.describe()

,fips,none,d0,d1,d2,d3,d4,duration
count,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10
mean,2013.0,100.0,0.0,0.0,0.0,0.0,0.0,6 days 00:00:00
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 00:00:00
min,2013.0,100.0,0.0,0.0,0.0,0.0,0.0,6 days 00:00:00
25%,2013.0,100.0,0.0,0.0,0.0,0.0,0.0,6 days 00:00:00
50%,2013.0,100.0,0.0,0.0,0.0,0.0,0.0,6 days 00:00:00
75%,2013.0,100.0,0.0,0.0,0.0,0.0,0.0,6 days 00:00:00
max,2013.0,100.0,0.0,0.0,0.0,0.0,0.0,6 days 00:00:00


In [34]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
q= '''
select chemical_species, count(*) as times_measured
from ch
group by 1
'''
chem_count= pysqldf(q)

In [37]:
chem_count.head()

,chemical_species,times_measured
0,Uranium,10


In [64]:
q= '''
select a.state, count(a.year) as num_years, avg(a.num) as avg_droughts, avg(a.dur/a.num) as avg_duration
from 
(select state, strftime('%Y',valid_start) as year, count(*) as num, sum(duration_int) as dur
from dr
group by 1, 2) a
group by 1 
'''
ad = pysqldf(q)
ad.head()

,state,num_years,avg_droughts,avg_duration
0,AK,1,10.0,6.0
